<a href="https://colab.research.google.com/github/lucprosa/dataeng-basic-course/blob/main/spark_streaming/examples/example_2_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Usecase 2
- Reading data from "rate"
- Aggregating data by window time
- Checking results from query in memory

# Setting up PySpark

In [ ]:
%pip install pyspark

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('Test streaming').getOrCreate()

# Write output in memory

In [4]:
import pyspark.sql.functions as F

# read stream
stream1 = spark.readStream.format("rate").option("rowsPerSecond", 10).load()

# transform
transformed = stream1.withColumn("minute", F.minute("timestamp"))
agg = transformed.groupBy(F.window(transformed.timestamp, "5 seconds")).count()

# write stream in memory
query = (agg.writeStream
.format('memory')
.queryName('my_query')
.outputMode('complete')
.start()
)

In [52]:
spark.sql("select * from my_query order by window desc").show(10,False)

+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|{2024-11-22 15:45:15, 2024-11-22 15:45:20}|47   |
|{2024-11-22 15:45:10, 2024-11-22 15:45:15}|50   |
|{2024-11-22 15:45:05, 2024-11-22 15:45:10}|50   |
|{2024-11-22 15:45:00, 2024-11-22 15:45:05}|50   |
|{2024-11-22 15:44:55, 2024-11-22 15:45:00}|50   |
|{2024-11-22 15:44:50, 2024-11-22 15:44:55}|50   |
|{2024-11-22 15:44:45, 2024-11-22 15:44:50}|50   |
|{2024-11-22 15:44:40, 2024-11-22 15:44:45}|50   |
|{2024-11-22 15:44:35, 2024-11-22 15:44:40}|50   |
|{2024-11-22 15:44:30, 2024-11-22 15:44:35}|50   |
+------------------------------------------+-----+
only showing top 10 rows



In [53]:
query.stop()

# Write output as json

In [131]:
!rm -rf content/output

In [132]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame

def save_parquet(df, batch_id):
  (df
   .withColumn("batch_id",F.lit(batch_id))
   .withColumn("load_time",F.current_timestamp())
   .write.mode("append")
   .parquet("content/output/rate_parquet")
  )

# read stream
stream1 = spark.readStream.format("rate").option("rowsPerSecond", 10).load()

# transform
transformed = stream1.withWatermark("timestamp", "5 seconds").withColumn("minute", F.minute("timestamp"))
agg = transformed.groupBy(F.window(transformed.timestamp, "5 seconds")).count()

# write stream as parquet with foreachBatch
query = (agg.writeStream
.option('checkpointLocation', 'content/output/checkpoint')
.trigger(processingTime='20 seconds')
.outputMode('append')
.foreachBatch(save_parquet)
.start()
)


In [143]:
result = spark.read.format("parquet").load("content/output/rate_parquet/")
result.sort(F.asc("window")).show(100, False)

+------------------------------------------+-----+--------+--------------------------+
|window                                    |count|batch_id|load_time                 |
+------------------------------------------+-----+--------+--------------------------+
|{2024-11-22 16:36:45, 2024-11-22 16:36:50}|17   |2       |2024-11-22 16:37:20.187321|
|{2024-11-22 16:36:50, 2024-11-22 16:36:55}|50   |2       |2024-11-22 16:37:20.187321|
|{2024-11-22 16:36:55, 2024-11-22 16:37:00}|50   |3       |2024-11-22 16:37:40.240769|
|{2024-11-22 16:37:00, 2024-11-22 16:37:05}|50   |3       |2024-11-22 16:37:40.240769|
|{2024-11-22 16:37:05, 2024-11-22 16:37:10}|50   |3       |2024-11-22 16:37:40.240769|
|{2024-11-22 16:37:10, 2024-11-22 16:37:15}|50   |4       |2024-11-22 16:38:00.20902 |
|{2024-11-22 16:37:15, 2024-11-22 16:37:20}|50   |4       |2024-11-22 16:38:00.20902 |
|{2024-11-22 16:37:20, 2024-11-22 16:37:25}|50   |4       |2024-11-22 16:38:00.20902 |
|{2024-11-22 16:37:25, 2024-11-22 16:37:30}

In [ ]:
query.stop()

# Enrich data with faker

In [145]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.8 MB/s eta 0:00:00


In [ ]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from faker import Faker

def enrich_df(df, batch_id) -> DataFrame:
  fake = Faker()
  new_columns = {
      'name': F.lit(fake.name()),
      'address': F.lit(fake.address()),
      'email': F.lit(fake.email()),
      'dob': F.lit(fake.date_of_birth()),
      'phone': F.lit(fake.phone_number())
  }
  df = df.withColumns(new_columns)
  return df

# read stream
df_stream = spark.readStream.format("rate").option("rowsPerSecond", 10).load()

# write stream
query = (df_stream.writeStream
.format('memory')
.queryName('enriched')
.outputMode('append')
.start()
)

In [ ]:
spark.sql("select * from enriched").show()

+--------------------+-----+----------+--------------------+--------------------+----------+------------+
|           timestamp|value|      name|             address|               email|       dob|       phone|
+--------------------+-----+----------+--------------------+--------------------+----------+------------+
|2024-11-19 20:32:...|    0|John Silva|21383 Davis Green...|isabelsalas@examp...|1942-01-04|775.239.3916|
|2024-11-19 20:32:...|    1|John Silva|21383 Davis Green...|isabelsalas@examp...|1942-01-04|775.239.3916|
|2024-11-19 20:32:...|    2|John Silva|21383 Davis Green...|isabelsalas@examp...|1942-01-04|775.239.3916|
|2024-11-19 20:32:...|    3|John Silva|21383 Davis Green...|isabelsalas@examp...|1942-01-04|775.239.3916|
|2024-11-19 20:32:...|    4|John Silva|21383 Davis Green...|isabelsalas@examp...|1942-01-04|775.239.3916|
|2024-11-19 20:32:...|    5|John Silva|21383 Davis Green...|isabelsalas@examp...|1942-01-04|775.239.3916|
|2024-11-19 20:32:...|    6|John Silva|21383 D

In [ ]:
query.stop()